In [1]:
import h5py
import numpy as np
import collections
import scipy.sparse as sp_sparse
import tables
import pandas as pd

In [2]:
with h5py.File("random.hdf5", "r") as f:
    data = f["default"]
    print(min(data))
    print(max(data))
    print(data[:15])

-3.611643776032326
3.2554430439487625
[ 0.91313708  1.06775138 -1.3677647  -0.52172047 -1.45567818 -1.56245745
 -0.34155252  0.43653753  1.08656216 -1.21240849 -0.94595649 -1.16313263
  1.18139558 -1.24606549  1.45170731]


In [3]:
with h5py.File("GSE115149_filtered_gene_bc_matrices_h5.h5") as f:
    data = f["hg19"]
    print(list(data))

['barcodes', 'data', 'gene_names', 'genes', 'indices', 'indptr', 'shape']


/home/lubianat/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


In [4]:

def get_matrix_from_h5(filename):
    with tables.open_file(filename, 'r') as f:
        mat_group = f.get_node(f.root, 'hg19')
        barcodes = f.get_node(mat_group, 'barcodes').read()
        print(barcodes)
        data = getattr(mat_group, 'data').read()
        indices = getattr(mat_group, 'indices').read()
        indptr = getattr(mat_group, 'indptr').read()
        shape = getattr(mat_group, 'shape').read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)
        gene_names = f.get_node(mat_group, 'gene_names').read()
    
        return barcodes, matrix, gene_names
 
filtered_matrix_h5 = "GSE115149_filtered_gene_bc_matrices_h5.h5"
barcodes, filtered_feature_bc_matrix, gene_names = get_matrix_from_h5(filtered_matrix_h5)

[b'AAACCTGCAAGCCGTC-1' b'AAACCTGCACAGACTT-1' b'AAACCTGCACAGCGTC-1' ...
 b'TTTGTCACATCCCACT-2' b'TTTGTCACATCCGCGA-2' b'TTTGTCATCAGGCCCA-2']


In [5]:
import scipy.sparse

msc_dataset = pd.DataFrame.sparse.from_spmatrix(filtered_feature_bc_matrix)


In [6]:
msc_dataset.columns = barcodes
msc_dataset.index = gene_names
msc_dataset.head(1)

,b'AAACCTGCAAGCCGTC-1',b'AAACCTGCACAGACTT-1',b'AAACCTGCACAGCGTC-1',b'AAACCTGCATGGATGG-1',b'AAACCTGGTAAATGAC-1',b'AAACCTGTCCTGCAGG-1',b'AAACGGGTCACAGTAC-1',b'AAACGGGTCTCACATT-1',b'AAACGGGTCTCGCATC-1',b'AAACGGGTCTCTTATG-1',...,b'TTTGCGCTCGACCAGC-2',b'TTTGCGCTCGCCTGTT-2',b'TTTGCGCTCTAACTGG-2',b'TTTGGTTCAAGGACAC-2',b'TTTGGTTCACCTTGTC-2',b'TTTGGTTGTTCACCTC-2',b'TTTGGTTTCAGATAAG-2',b'TTTGTCACATCCCACT-2',b'TTTGTCACATCCGCGA-2',b'TTTGTCATCAGGCCCA-2'
b'MIR1302-10',0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
msc_dataset.to_csv("msc_dataset.csv")

KeyboardInterrupt: 